# To Visualize and Analysze tomorrow

In [1]:
import numpy as np
import tensorflow as tf
import os
import cv2
import pandas as pd
SEED = 1234

In [2]:
# Creating the training set
dataset_dir = os.getcwd() # Obtain 
dataset_dir = os.path.join(dataset_dir,'MaskDataset')
print(dataset_dir)
train_dir = os.path.join(dataset_dir, 'training')

classes = ["nomask","allmask","someone"]

# Create generators to read images from dataset directory
# -------------------------------------------------------

# Batch size
bs = 8

# img shape
img_h = 256
img_w = 256

num_classes = 3

# for image in os.path.listdir():  # all the images in the folder



c:\Users\enric\Downloads\AN2DL-1st-Project\MaskDataset


## Making the Image Generator

In [3]:
# ImageDataGenerator, it is a class
# ------------------
# Generate batches of tensor image data with real-time data augmentation.
# you have to prepare/create the dataset into a specific format that keras expects

#The data will be looped over (in batches)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255)

# Create validation and test ImageDataGenerator objects
valid_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)

## Importing the Json file and creating a dataframe

In [4]:

import json
with open(os.path.join(dataset_dir,"train_gt.json")) as f:
  dic = json.load(f)

dataframe = pd.DataFrame(dic.items())  # putting all the 
dataframe.rename(columns = {0:'filename', 1:'class'}, inplace = True)
dataframe["class"] = dataframe["class"].astype(str)


## Creating the Training Set

In [5]:
# Creating the training set for the images ! 
train_gen = []
train_gen = train_data_gen.flow_from_dataframe(dataframe,
                                               train_dir,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED) 

Found 5614 validated image filenames belonging to 3 classes.


In [6]:
# Test  
test_dir = os.path.join(dataset_dir, 'test')
test_gen = []
test_gen = test_data_gen.flow_from_directory(test_dir,
                                             batch_size=bs, 
                                             classes=classes,
                                             class_mode='categorical',
                                             shuffle=False,
                                             seed=SEED)

Found 0 images belonging to 3 classes.


In [7]:
# Check how keras assigned the labels
train_gen.class_indices

{'0': 0, '1': 1, '2': 2}

In [8]:
# Create Dataset objects
# ----------------------

# Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Shuffle (Already done in generator..)
# train_dataset          train_dataset.shuffle(buffer_size=len(train_gen))

# Normalize images (Already done in generator..)
# def normalize_img(x_, y_):
#     return tf.cast(x_, tf.float32) / 255., y_

# train_dataset = train_dataset.map(normalize_img)

# 1-hot encoding <- for categorical cross entropy (Already done in generator..)
# def to_categorical(x_, y_):
#     return x_, tf.one_hot(y_, depth=10)

# train_dataset = train_dataset.map(to_categorical)

# Divide in batches (Already done in generator..)
# train_dataset = train_dataset.batch(bs)

# Repeat
# Without calling the repeat function the dataset 
# will be empty after consuming all the images
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
valid_dataset = valid_dataset.repeat()

# Test
# ----
test_dataset = tf.data.Dataset.from_generator(lambda: test_gen,
                                              output_types=(tf.float32, tf.float32),
                                              output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
test_dataset = valid_dataset.repeat()

In [9]:
# Architecture: Features extraction -> Classifier

start_f = 8
depth = 5

model = tf.keras.Sequential()

# Features extraction
for i in range(depth):

    if i == 0:
        input_shape = [img_h, img_w, 3]
    else:
        input_shape=[None]

    # Conv block: Conv2D -> Activation -> Pooling
    model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                     kernel_size=(3, 3),
                                     strides=(1, 1),
                                     padding='same',
                                     input_shape=input_shape))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

    start_f *= 2
    
# Classifier
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [10]:
# Visualize created model as a table
model.summary()

# Visualize initialized weights
model.weights

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 8)       224       
_________________________________________________________________
re_lu (ReLU)                 (None, 256, 256, 8)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 16)      1168      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 128, 128, 16)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        4

 ..., -0.09604548,
            0.05065382, -0.05381875],
          ...,
          [-0.09340868, -0.05904023, -0.06751709, ...,  0.11016706,
            0.02826837, -0.01356491],
          [ 0.07285238,  0.11340944,  0.04960083, ...,  0.06572748,
            0.03836184,  0.01195153],
          [-0.05857274, -0.09475148, -0.02094398, ..., -0.11464626,
            0.01696371, -0.1177724 ]],
 
         [[ 0.11038395, -0.04240464, -0.02834746, ..., -0.00973219,
           -0.09951261,  0.0839253 ],
          [ 0.07607689, -0.06663667, -0.03701645, ..., -0.02730579,
           -0.04417448, -0.05067246],
          [-0.11213282, -0.08718655, -0.11578265, ..., -0.10750231,
            0.04354531,  0.07315602],
          ...,
          [ 0.07940272,  0.03030435,  0.03085352, ..., -0.00862794,
           -0.07668498, -0.00444245],
          [ 0.08140945,  0.07653978, -0.00133954, ...,  0.04740607,
            0.11743129,  0.01804259],
          [ 0.11064068,  0.03804464,  0.1094432 , ...,  0.1035

In [11]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [12]:
model.fit(x=train_dataset,
          epochs=17,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen))

# If you can get "Failed to get convolution algorithm. This is probably because cuDNN failed to initialize", your GPU is full ! Restart all kernels ! 

# How to visualize Tensorboard

# 1. tensorboard --logdir EXPERIMENTS_DIR --port PORT     <- from terminal
# 2. localhost:PORT   <- in your browser

Train for 702 steps
Epoch 1/13
 62/702 [=>............................] - ETA: 1:39 - loss: 1.1156 - accuracy: 0.3094

KeyboardInterrupt: 

## Visualize a Random Image

In [13]:
# Let's visualize the activations of our network
from PIL import Image

test_iter = iter(test_dataset)

# Get a test image
test_img = next(test_iter)[0]
test_img = test_img[0]

# Visualize the image
Image.fromarray(np.uint8(np.array(test_img)*255.))

UnknownError: NameError: name 'valid_gen' is not defined
Traceback (most recent call last):

  File "C:\Users\enric\anaconda3\envs\tensorflow_gpuenv\lib\site-packages\tensorflow_core\python\data\ops\dataset_ops.py", line 673, in get_iterator
    return self._iterators[iterator_id]

KeyError: 0


During handling of the above exception, another exception occurred:


Traceback (most recent call last):

  File "C:\Users\enric\anaconda3\envs\tensorflow_gpuenv\lib\site-packages\tensorflow_core\python\ops\script_ops.py", line 236, in __call__
    ret = func(*args)

  File "C:\Users\enric\anaconda3\envs\tensorflow_gpuenv\lib\site-packages\tensorflow_core\python\data\ops\dataset_ops.py", line 789, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "C:\Users\enric\anaconda3\envs\tensorflow_gpuenv\lib\site-packages\tensorflow_core\python\data\ops\dataset_ops.py", line 675, in get_iterator
    iterator = iter(self._generator(*self._args.pop(iterator_id)))

  

In [15]:
# Get the activations (the output of each ReLU layer)
layer_outputs = [layer.output for layer in model.layers if isinstance(layer, tf.keras.layers.ReLU)]
# We can do it by creating a new model (activation_model) with model.input as input 
# and all the ReLU activations as output
activation_model = tf.keras.Model(inputs=model.input, outputs=layer_outputs)
# Finally we get the output values given the imput test image
activations = activation_model.predict(tf.expand_dims(test_img, 0))

NameError: name 'test_img' is not defined

## Evaluation of the Model

In [ ]:
# Print Confusion Matrix and Classification Report (Precision, Recall, and F1-score)
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

Y_prediction = model.predict_generator(test_gen, len(test_gen))
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_prediction,axis = 1) 
actual_classes = 0 # no becuase we don't know the test labels
# Convert validation observations to one hot vectors
Y_true = test_gen.classes
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
class_report = classification_report(Y_true, Y_pred_classes, 
                                     target_names=test_gen.class_indices.keys())  # target_names must be ordered depending on the class labels
print('Confusion Matrix:')
print(confusion_mtx)
print()
print('Classification Report:')
print(class_report)

In [2]:
def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [4]:
dataset_dir = "C:/Users/enric/Downloads/AN2DL-1st-Project/MaskDataset"
test_dir = os.path.join(dataset_dir, 'test')

images = [f for f in listdir(test_dir)]
images = pd.DataFrame(images)
images.rename(columns = {0:'filename'}, inplace = True)
images["class"] = 'test'

test_gen = train_data_gen.flow_from_dataframe(images,
                                               test_dir,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)


test_gen.reset()

predictions = model.predict_generator(test_gen, len(test_gen), verbose=1)

results = {}
images = test_gen.filenames
i = 0



for p in predictions:
  prediction = np.argmax(p)
  import ntpath
  image_name = ntpath.basename(images[i])
  results[image_name] = str(prediction)
  i = i + 1

NameError: name 'listdir' is not defined

## Evaluation of the model.

In [ ]:
# Print Confusion Matrix and Classification Report (Precision, Recall, and F1-score)
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

Y_prediction = model.predict_generator(test_gen, len(test_gen))
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_prediction,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = test_gen.classes
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
class_report = classification_report(Y_true, Y_pred_classes, 
                                     target_names=test_gen.class_indices.keys())  # target_names must be ordered depending on the class labels
print('Confusion Matrix:')
print(confusion_mtx)
print()
print('Classification Report:')
print(class_report)